In [ ]:
"""
RNN
 - 순서(이미지와 같은)와 시간성(시계열분석)을 가진데이터에서 특징을 추출하는양
 - rnn을 구성하는 cell하나는 ffnn망으로 구성된다
 - cell과 cell을 연결해서 시간적으로 누적되는 특성을 추출한다
 - rnn은 다양한 망 구성이 가능하다 (압력과 출력의 구성으로 )
      - one to one : 바닐라망( 기본망 )
      - one to many : captioning
      - many to one : 감정분석
      - many to many : 번역망이나 비디오 해석
      
"""

In [2]:
import numpy as np
import os
import tensorflow as tf
tf.reset_default_graph()
n_inputs = 3 # 들어온다 1x3 # 셀당 입력 차수
n_neurons = 5 # 나간다 1x5 필요한 가중치 3x5 # 셀당 출력 차수
X0 = tf.placeholder(tf.float32, [None, n_inputs]) # None == 4, 4x3
X1 = tf.placeholder(tf.float32, [None, n_inputs])

# 셀정의
# 셀당 필요한 가중치 3x5
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)#출력차수

# rnn망 구성 : static_rnn(메모리를 확보), dynamic_rnn(동적으로 메모리확보)
# 입력 2x4x3 # 셀수 4
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, [X0, X1], dtype=tf.float32) # 2 

# 출력 output_seqs = 2x4x5 
# state = 1x5
Y0, Y1 = output_seqs # 각각 4x5

init = tf.global_variables_initializer()

# 번역 : 나는 학교에 간다 => encoding
# A ascii code : 65
X0_batch = np.array([[0,1,2], [3,4,5], [6,7,8], [9,0,1]]) # 입력 4x3 # 아스키
X1_batch = np.array([[9,8,7], [0,0,0], [6,5,4], [3,2,1]])
with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch}) # 처리결과 나온 시퀀스
print("처음데이터 특성 : \n", Y0_val, "차수 : ", Y0_val.shape)
print("두번째데이터 특성 : \n", Y1_val, "차수 : ", Y1_val.shape)

처음데이터 특성 : 
 [[ 0.21256007  0.5256222  -0.3534151  -0.91985595 -0.8965427 ]
 [ 0.99056077  0.7047399  -0.74324226 -0.999991   -0.9868129 ]
 [ 0.9999307   0.82402647 -0.91312766 -1.         -0.998386  ]
 [ 0.9988971  -0.99174494 -0.9281864  -0.9999409   0.9999681 ]] 차수 :  (4, 5)
두번째데이터 특성 : 
 [[ 0.99999905 -0.79623216 -0.95116675 -1.         -0.99529076]
 [-0.06942188 -0.92542714 -0.4515368   0.2824346  -0.84856373]
 [ 0.99983263 -0.932675   -0.86080503 -0.99999905 -0.96854764]
 [ 0.9750643   0.27966166 -0.6391409  -0.98708534  0.84477115]] 차수 :  (4, 5)


In [6]:
"""
다음 모델에서 차수들을 결정하시오
"""
import numpy as np
import tensorflow as tf
tf.reset_default_graph()
n_steps = 2 # 셀수 
n_inputs = 3 # 셀당입력
n_neurons = 5 # 셀당출력
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs]) # None : 4
basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)

# static : for 문을 사용하지 않고 메모리 공간확보
# dynamic : for문을 이용해서 계산
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

# outputs : 4x2x5, states : 1x5
X_batch = np.array([ # 4x2x3 # 3:셀당입력 # 2:셀수 # 4:배치사이즈
    [[0,1,2], [9,8,7]],
    [[3,4,5], [3,4,5]],
    [[6,7,8], [6,5,4]],
    [[9,0,1], [3,2,1]],
])
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    output_val = outputs.eval(feed_dict={X:X_batch})
print("output_val : {}\n{}".format(output_val.shape, output_val))



output_val : (4, 2, 5)
[[[ 0.16713928 -0.23167202 -0.55705845  0.87856007  0.50833905]
  [ 0.93700606 -0.9851856  -0.9999999   1.          0.9999807 ]]

 [[ 0.74557114 -0.8792165  -0.99913025  0.99997157  0.98876274]
  [-0.02421216 -0.0790673  -0.9905376   0.99997735  0.99517924]]

 [[ 0.9421775  -0.9868425  -0.99999857  1.          0.99980414]
  [ 0.35380256 -0.5951678  -0.9997788   0.9999991   0.9992617 ]]

 [[-0.53928727  0.54787964 -0.9991711   0.9998946   0.95357466]
  [ 0.36374104 -0.41695455 -0.9587238   0.8786859   0.91251093]]]


In [126]:
import matplotlib.pyplot as plt
tf.reset_default_graph() # 그래프 초기화
n_steps = 28 # 셀수
n_inputs = 28 # 셀당입력
    # 28x28 => 이미지한장
n_neurons = 150 # 셀당출력
    # 특징확대
n_outputs = 10 # 
learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs]) # ?, 28, 28
y = tf.placeholder(tf.int32, [None]) # ?

#RNN망 구성및 특징 추출
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons) # 28 => 150
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32) # ?(밑코드 150), 28, 28
# outputs : ?x28x150 , states : ? x 150

#FFNN 시작
#가중치 다 생성
logits = tf.layers.dense(states, n_outputs) # fully-connected : 가중치 생략
# 가중치를 계산 : ?(밑코드 150)x150 -> 150x10(학습해서 찾은 가중치) -> 150x10
# cost function : cross_entropy
# 확률값으로 변환 -> 비용값을 계산
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)

loss = tf.reduce_mean(xentropy) # 비용 : 배치사이즈에 대한 평균값으로
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss) # 학습
correct = tf.nn.in_top_k(logits, y, 1) # 가장 큰수의 인덱스구하고 y(실제값)과 비교
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32)) # 정확도 # 캐스팅:확률값은 소수점까지 계산
init = tf.global_variables_initializer()


In [127]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data/')
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs)) # 원래의 이미지 사이즈
y_test = mnist.test.labels
n_epochs = 100
batch_size = 150
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        # 60000 / 150
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            #-1 : 계산결과 나머지 : % 150
            X_batch = X_batch.reshape((-1, n_steps, n_inputs)) # 원래의 이미지 사이즈
            
            # 150x28x28
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch}) # 학습
        acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch}) # 정확도
        acc_test = accuracy.eval(feed_dict={X:X_test, y:y_test}) #
        print(epoch, "Train accuracy:", acc_train, "Test_accurracy:", acc_test)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
0 Train accuracy: 0.93333334 Test_accurracy: 0.9257
1 Train accuracy: 0.96 Test_accurracy: 0.95
2 Train accuracy: 0.96666664 Test_accurracy: 0.9575
3 Train accuracy: 0.97333336 Test_accurracy: 0.9601
4 Train accuracy: 0.98 Test_accurracy: 0.9619
5 Train accuracy: 0.96 Test_accurracy: 0.9613
6 Train accuracy: 0.96 Test_accurracy: 0.9687
7 Train accuracy: 0.96 Test_accurracy: 0.9605
8 Train accuracy: 0.99333334 Test_accurracy: 0.9664
9 Train accuracy: 0.96 Test_accurracy: 0.9662
10 Train accuracy: 0.97333336 Test_accurracy: 0.9711
11 Train accuracy: 0.98 Test_accurracy: 0.9738
12 Train accuracy: 0.98 Test_accurracy: 0.9672
13 Train accuracy: 0.96666664 Test_accurracy: 0.9709
14 Train accuracy: 0.9866667 Test_accurracy: 0.9749
15 Train accuracy: 0.99333334 Test_accurracy: 0.9748
16 Train accuracy: 0.98

In [13]:
# encoding -> decoding
# 끝글자 예측 
tf.reset_default_graph()
import tensorflow as tf
import numpy as np

# 알파벳
# 알파벳을 숫자로 인덱싱 -> 숫자를 문자로 변경가능
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

# 글자에 대한 인덱스 번호 구성
# a:0 b:1 c:2 ... z:25
# 문자를 숫자로 변경
num_dic = {n: i for i, n in enumerate(char_arr)} # 순서대로 번호지정

dic_len = len(num_dic) #26
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind'] 

In [16]:
print(np.eye(26)[22]) # w
print(np.eye(26)[14]) # o
print(np.eye(26)[17]) # r 
print(np.eye(26)[3]) # d

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


In [14]:
# 원핫인코딩 : 신경망에서 범주형의 경우 원핫인코딩

# 셀수 3
# 셀당입력 26
def make_batch(seq_data):
    input_batch = [] # 3글자
    target_batch = [] # 1글자
    for seq in seq_data: # word
        input = [num_dic[n] for n in seq[:-1]] # wor -> w:22, o:14, r:17 인코딩
        target = num_dic[seq[-1]] # d:3
        # 신경망에서 범주형 데이터는 원핫인코딩되야한다
        # eye : 단위행렬 : 대각선1
        input_batch.append(np.eye(dic_len)[input])# 단위행렬을 이용한 원핫원코딩
        target_batch.append(target) # 3
    return input_batch, target_batch

In [31]:
learning_rate = 0.01
n_hidden = 128 # 가중치를 위한 히든
total_epoch = 30
n_step = 3 # 셀수
n_input = n_class = dic_len # 26(a~z)
X = tf.placeholder(tf.float32, [None, n_step, n_input]) # 배치사이즈, 셀수, 셀당입력
Y = tf.placeholder(tf.int32, [None])

# FFNN을 위한 가중치 준비
W = tf.Variable(tf.random_normal([n_hidden, n_class])) # 128x26
b = tf.Variable(tf.random_normal([n_class]))

# rnn으로 특징 추출( 셀당 가중치가 학습 )
# control_state를 추가 : 장기기억이 가능하도록 : 변종이 많음
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden) # 나가는 차수

# 일부 계산회로 줄임 ( 랜덤 ) : 과적합 때문에
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5, seed=10)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

# multiRNN : 상하로 RNN셀을 두번 쌓음( 더 정확한 학습 ) 
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])  
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32) # ?x3x26 # 셀 3개가 결정
# 배치사이즈x3x128(히든), 배치사이즈x128
outputs = tf.transpose(outputs, [1,0,2]) # 3x배치사이즈x128
outputs = outputs[-1] # 맨뒤  # 맨마지막셀  # 배치사이즈x128
model = tf.matmul(outputs, W)+b # == dense와 동일하게 연산 : 배치사이즈x26
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))

# RMSProp + momentum을 조정
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
print('-')

-


In [ ]:
# 오전 종료 ===========================================================================================================================================================================================================

In [32]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)# 변수둘 - 원핫인코딩 결과, 문자를 숫자로매핑
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], #optimizer는 학습, cost는 에포크당 비용 출력
                       feed_dict={X: input_batch, Y:target_batch})
    print('Epoch', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))    
    
# model은 분산과 같은 형태로 예측
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32)) # 정확도
input_batch, target_batch = make_batch(seq_data)

# predict : 숫자 -> 디코딩
predict, accuracy_val = sess.run([prediction, accuracy], feed_dict={X: input_batch, Y:target_batch})
predict_words = []

for idx, val in enumerate(seq_data):
    # target의 알파벳을 번호로 결정
    last_char = char_arr[predict[idx]] # inverse mapping 문자 -> 숫자, 숫자 -> 문자
    predict_words.append(val[:3] + last_char) # 입력된 데이터 + 예측된 끝단어
print("\n==예측결과==")
print("입력값 : ", [w[:3]+ ' ' for w in seq_data])
print('예측값 : ', predict_words)
print('정확도 : ', accuracy_val)

Epoch 0001 cost = 4.531738
Epoch 0002 cost = 3.467937
Epoch 0003 cost = 1.893429
Epoch 0004 cost = 1.725192
Epoch 0005 cost = 0.934616
Epoch 0006 cost = 0.798401
Epoch 0007 cost = 0.774154
Epoch 0008 cost = 0.728611
Epoch 0009 cost = 0.643163
Epoch 0010 cost = 0.572102
Epoch 0011 cost = 0.245229
Epoch 0012 cost = 0.477978
Epoch 0013 cost = 0.388104
Epoch 0014 cost = 0.248850
Epoch 0015 cost = 0.110863
Epoch 0016 cost = 0.086742
Epoch 0017 cost = 0.235641
Epoch 0018 cost = 0.107952
Epoch 0019 cost = 0.196902
Epoch 0020 cost = 0.141469
Epoch 0021 cost = 0.118111
Epoch 0022 cost = 0.054736
Epoch 0023 cost = 0.033359
Epoch 0024 cost = 0.077499
Epoch 0025 cost = 0.025251
Epoch 0026 cost = 0.013885
Epoch 0027 cost = 0.043071
Epoch 0028 cost = 0.065216
Epoch 0029 cost = 0.018888
Epoch 0030 cost = 0.037207

==예측결과==
입력값 :  ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값 :  ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도 

In [128]:
"""
번역망 ( seq 2 seq )
"""
#email
import tensorflow as tf
import numpy as np
from tensorflow.python.framework import ops
ops.reset_default_graph()

char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀연습사랑']

num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)
print(dic_len)
seq_data = [['word', '단어'], ['wood', '나무'],
            ['game', '놀이'], ['girl', '소녀'],
            ['test', '연습'], ['love', '사랑']] 

# ==
# 왼쪽망   : input(영어) => smoking gun, 
# 오른쪽망 : output(한), target(한)
# 두 개의 망은 연결

def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []
    for seq in seq_data:
        input = [num_dic[n] for n in seq[0]] # w:25 o:17 r:20 d:6
        output = [num_dic[n] for n in ('S' + seq[1])] # SEP : start: 시작의 의미 비어있음 S:0 29:단 30:어
        target = [num_dic[n] for n in (seq[1] + 'E')] # SEP : end: 번역의 종료지점(계속번역금지-공간) 29:단 30:어 E:1
        input_batch.append(np.eye(dic_len)[input]) # 원핫인코딩
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target) # [0, 29, 30]
    return input_batch, output_batch, target_batch
        
        

41


In [129]:
tf.reset_default_graph()
learning_rate = 0.01
n_hidden = 128 # 셀당출력
total_epoch = 100
n_class = n_input = dic_len # 41
enc_input = tf.placeholder(tf.float32, [None, None, n_input]) # 셀당입력x셀수x배치 : 6x4(알파벳이 4개)x41
dec_input = tf.placeholder(tf.float32, [None, None, n_input]) # 6x3x41
targets = tf.placeholder(tf.int64, [None, None]) # 6x3

# 연결지점을 확인하시오
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden) # 출력차수 128
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5, seed=100)
    
    # 셀수 4개로 결정
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)
    
# output : 6x4x128
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden) # 128
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    
    # 셀이 3개로 결정
    # 연결부 : initial_state
    outputs, enc_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state=enc_states, dtype=tf.float32) 
    
# output : 6x3x128
model = tf.layers.dense(outputs, n_class, activation=None) # 3x41
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('-')

input_batch, output_batch, target_batch = make_batch(seq_data)
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                      feed_dict={enc_input: input_batch,
                                dec_input: output_batch,
                                targets : target_batch})
    print('Epoch: ', '%04d' % (epoch + 1), 'cost = ', '{:.6f}'.format(loss) )
print('-')

-
Epoch:  0001 cost =  3.780606
Epoch:  0002 cost =  2.590755
Epoch:  0003 cost =  1.767363
Epoch:  0004 cost =  1.220510
Epoch:  0005 cost =  0.869404
Epoch:  0006 cost =  0.471994
Epoch:  0007 cost =  0.220139
Epoch:  0008 cost =  0.205051
Epoch:  0009 cost =  0.125327
Epoch:  0010 cost =  0.091335
Epoch:  0011 cost =  0.071131
Epoch:  0012 cost =  0.186076
Epoch:  0013 cost =  0.071023
Epoch:  0014 cost =  0.029767
Epoch:  0015 cost =  0.048297
Epoch:  0016 cost =  0.052502
Epoch:  0017 cost =  0.017864
Epoch:  0018 cost =  0.039752
Epoch:  0019 cost =  0.005683
Epoch:  0020 cost =  0.013886
Epoch:  0021 cost =  0.009827
Epoch:  0022 cost =  0.003929
Epoch:  0023 cost =  0.004667
Epoch:  0024 cost =  0.023378
Epoch:  0025 cost =  0.002879
Epoch:  0026 cost =  0.003372
Epoch:  0027 cost =  0.012189
Epoch:  0028 cost =  0.002995
Epoch:  0029 cost =  0.006330
Epoch:  0030 cost =  0.004249
Epoch:  0031 cost =  0.009292
Epoch:  0032 cost =  0.007687
Epoch:  0033 cost =  0.001981
Epoch:  

In [130]:
def translate(word):
    seq_data = [word, 'P' * len(word)] # 번역된 데이터 생성 ( 임의의 글자를 P )
    input_batch, output_batch, target_batch = make_batch([seq_data])
    prediction = tf.argmax(model, 2) # 예측값 6x3x41 (41개의 단어중 하나로)
    result = sess.run(prediction, feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch})
    
    # 번역할 단어 41개중 하나로 예측
    decoded = [char_arr[i] for i in result[0]] # 숫자를 문자로 매핑
    end = decoded.index('E') # E를 예측
    translated = ''.join(decoded[:end]) # E앞단까지 단어 조합
    return translated

print('word -> ', translate('word'))
print('love -> ', translate('love'))
print('abcd -> ', translate('abcd'))


print('wodr -> ', translate('wodr'))
print('loev -> ', translate('loev'))

word ->  단어
love ->  사랑
abcd ->  나무단
wodr ->  나무
loev ->  사랑


In [132]:
"""
문제 : 다음 인코딩 된 값을 2개의 특성을 가진 데이터로 변환해 보시오
h=[1,0,0,0]
e=[0,1,0,0]
l=[0,0,1,0]
o=[0,0,0,1]
# hello 데이터에 대하여 2개의 특성으로 추출하시오
# 5x2로 출력되도록 해보시오

결정해야될것 : 배치사이즈, 셀수, 셀당출력(아웃풋) : 1, 5, 2
"""

In [188]:
ops.reset_default_graph()
from tensorflow.contrib import rnn
import pprint
sess = tf.InteractiveSession()

h=[1,0,0,0]
e=[0,1,0,0]
l=[0,0,1,0]
o=[0,0,0,1]

pp = pprint.PrettyPrinter(indent=4)

with tf.variable_scope('hello') as scope: # 클로즈 자동호출 with # 그래프내부 네임스페이스
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    x_data = np.array([[h, e, l, l, o]], dtype=np.float32) # 5x4
    print(x_data.shape)
    outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    print(outputs.shape)
    print(states.shape)
    pp.pprint(states.eval())
sess.close()


(1, 5, 4)
(1, 5, 2)
(1, 2)
array([[-0.63022363,  0.6946867 ]], dtype=float32)


In [189]:
"""
3 batches 'hello', 'eolll', 'lleel' 2개의 특성으로 embedding 해보시오
"""
sess = tf.InteractiveSession()


x_data = np.array([[h,e,l,l,o], 
                   [e,o,l,l,l], 
                   [l,l,e,e,l]], dtype=np.float32)


pp.pprint(x_data)

with tf.variable_scope('hello2') as scope:
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)


    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
sess.close()

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-0.6651914 , -0.6653228 ],
        [-0.4614807 ,  0.4314423 ],
        [-0.2528372 , -0.86558074],
        [-0.602575  , -0.12148489],
        [-0.60226464, -0.6724281 ]],

       [[-0.29223335,  0.31307113],
        [-0.51014775, -0.7536512 ],
        [-0.5723368 , -0.35576442],
        [-0.47496527, -0.62727904],
        [-0.5442506 , -0.42532873]],

       [[-0.3952672 , -0.58989143],
        [-0.53741306, -0.41026315],
        [-0.3959082 ,  0.31917316],
        [-0.17909819, -0.17932078],
        [-0.43936962, -0.5650803 ]]], dtype=float32)


In [169]:
# 텍스트마이닝
from konlpy.tag import Okt
okt=Okt()
token=okt.morphs('대한민국의 무궁한 발전과 세계를 이끄는 지도 국가가 되길 희망합니다.')
print(token)

['대한민국', '의', '무궁', '한', '발전', '과', '세계', '를', '이끄는', '지도', '국가', '가', '되길', '희망', '합니다', '.']


In [165]:
# 원핫 인코딩
word2index={}
for voca in token:
    if voca not in word2index.keys():
        word2index[voca] = len(word2index) # 키 데이터
print(word2index)

def one_hot_encoding(word, word2index):
    one_hot_vector = [0] * (len(word2index))
    index = word2index[word]
    one_hot_vector[index]=1 # 지정단어 원핫 인코딩
    return one_hot_vector
one_hot_encoding("세계", word2index)


{'대한민국': 0, '의': 1, '무궁': 2, '한': 3, '발전': 4, '과': 5, '세계': 6, '를': 7, '이끄는': 8, '지도': 9, '국가': 10, '가': 11, '되길': 12, '희망': 13, '합니다': 14, '.': 15}


[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]